## Structured Tool Output

Another important addition in the 2025-06-18 spec is support for **structured tool output**.
Previously, tool results were allowed to contain structured data but the host/LLM had to perform the parsing and interpretation
without any guidance from the tool itself.
Now, tools can return structured content that is explicitly defined, allowing AI models to better understand and process the output.



The C# SDK supports this by allowing tools to specify that their output is structured, with the [UseStructuredContent] parameter of the [McpServerTool] attribute.

[McpServerTool]: https://modelcontextprotocol.github.io/csharp-sdk/api/ModelContextProtocol.Server.McpServerToolAttribute.html
[UseStructuredContent]: https://modelcontextprotocol.github.io/csharp-sdk/api/ModelContextProtocol.Server.McpServerToolAttribute.html#ModelContextProtocol_Server_McpServerToolAttribute_UseStructuredContent

```csharp
[McpServerTool(UseStructuredContent = true), Description("Gets a list of structured product data with detailed information.")]
public static List<Product> GetProducts(int count = 5)
```



The C# SDK will generate a JSON schema for the tool's output based on the return type of the method
and will include this schema in the tool's metadata. Here is an example of the response to a `tools/list` call
that shows the output schema for the `get_products` tool:

In [1]:
curl -s -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-d '{
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list"
}' | Where-Object { $_ -like "data:*" } | ForEach-Object { $_.Substring(5) } | jq

{
  "result": {
    "tools": [
      {
        "name": "get_products",
        "description": "Gets a list of structured product data with detailed information.",
        "inputSchema": {
          "type": "object",
          "properties": {
            "count": {
              "type": "integer",
              "default": 5
            }
          }
        },
        "outputSchema": {
          "type": "object",
          "properties": {
            "result": {
              "type": "array",
              "items": {
                "type": "object",
                "properties": {
                  "id": {
                    "description": "Unique identifier for the product",
                    "type": "integer"
                  },
                  "name": {
                    "description": "Name of the product",
                    "type": "string"
                  },
                  "description": {
                    "description": "Detailed description of the product",
  


And when the tool is called, the tool response will include the structured output in the `result.structuredContent` field:

In [2]:
curl -s -X POST http://localhost:3001 `
-H "ProtocolVersion: 2025-06-18" `
-H "Accept: application/json, text/event-stream" `
-H "Content-Type: application/json" `
-d '{
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "get_products"
    }
}' | Where-Object { $_ -like "data:*" } | ForEach-Object { $_.Substring(5) } | jq

{
  "result": {
    "content": [
      {
        "type": "text",
        "text": "[{\"id\":1,\"name\":\"Laptop Pro\",\"description\":\"High-quality laptop pro for professional use\",\"price\":252.41,\"category\":\"Hardware\",\"brand\":\"TechCorp\",\"inStock\":50,\"rating\":3,\"features\":[\"Durable construction\",\"Modern design\",\"Easy to use\"],\"specifications\":{\"Weight\":\"3 lbs\",\"Dimensions\":\"16x10x2 inches\",\"Warranty\":\"2 years\"}},{\"id\":2,\"name\":\"Wireless Mouse\",\"description\":\"High-quality wireless mouse for professional use\",\"price\":347.01,\"category\":\"Accessories\",\"brand\":\"TechCorp\",\"inStock\":95,\"rating\":4.1,\"features\":[\"Durable construction\",\"Modern design\",\"Easy to use\"],\"specifications\":{\"Weight\":\"3 lbs\",\"Dimensions\":\"19x9x4 inches\",\"Warranty\":\"2 years\"}},{\"id\":3,\"name\":\"Mechanical Keyboard\",\"description\":\"High-quality mechanical keyboard for professional use\",\"price\":484.62,\"category\":\"Electronics\",\"br